In [0]:
%scala
val storageAccount = "grupo" //nombre de la cuenta de almacenamiento 
val container = "grupo5" //nombre del contenedor
val sasKey = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T22:58:48Z&st=2024-12-07T14:58:48Z&spr=https&sig=65DfM0jHl5fWF83LxT3Sbm%2B2vr6dxmLAglPWn4FDnCA%3D" //llave de contenedor
val conf = "fs.azure.sas." + container + "." + storageAccount + ".blob.core.windows.net" 
//crear acceso directo
dbutils.fs.mount(source="wasbs://" + container + "@" + storageAccount + ".blob.core.windows.net", mountPoint="/mnt/" + container, extraConfigs=Map(conf -> sasKey))

storageAccount: String = grupo
container: String = grupo5
sasKey: String = sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T22:58:48Z&st=2024-12-07T14:58:48Z&spr=https&sig=65DfM0jHl5fWF83LxT3Sbm%2B2vr6dxmLAglPWn4FDnCA%3D
conf: String = fs.azure.sas.grupo5.grupo.blob.core.windows.net
res0: Boolean = true

In [0]:
%fs ls mnt/grupo5

path,name,size,modificationTime
dbfs:/mnt/grupo5/proyectofinal/,proyectofinal/,0,0


In [0]:
##Ingestamos archivo Producto.parquet
producto = (spark.read.parquet("/mnt/grupo5/proyectofinal/capaplata/Producto.parquet"))
producto.createOrReplaceTempView("s_producto")

In [0]:
##Ingestamos archivo Categoria.parquet
categoria = (spark.read.parquet("/mnt/grupo5/proyectofinal/capaplata/Categoria.parquet"))
categoria.createOrReplaceTempView("s_categoria")

In [0]:
##Ingestamos archivo SubCategoria.parquet
subcategoria = (spark.read.parquet("/mnt/grupo5/proyectofinal/capaplata/SubCategoria.parquet"))
subcategoria.createOrReplaceTempView("s_subcategoria")

In [0]:
%sql
--Eliminar tabla en caso exista
DROP TABLE IF EXISTS Fact_Product;
--Crear tabla
CREATE TEMP VIEW Fact_Product
AS
SELECT P.Cod_Producto, P.Nombre AS Producto, S.Nombre AS SubCategoria, C.Nombre AS Categoria
FROM s_producto P
JOIN s_subcategoria S ON P.Cod_SubCategoria = S.Cod_SubCategoria
JOIN s_categoria C ON S.Cod_Categoria = C.Cod_Categoria

In [0]:
dfFactProducto = spark.table("Fact_Product")
dfFactProducto.write.mode("overwrite").parquet("/mnt/grupo5/proyectofinal/capaoro/FactProducto.parquet")

In [0]:
#Eliminar acceso directo
dbutils.fs.unmount("/mnt/grupo5")

/mnt/grupo5 has been unmounted.


True